In [1]:
import requests
import json
import pandas as pd
import random
baseURL="http://b.libtech.in"
apiusername='apimanager'
apipassword='sharedata'
AUTHENDPOINT='%s/auth/jwt/' % (baseURL)
TESTURL="%s/api/test/" % (baseURL)
REPORTURL="%s/api/report/" % (baseURL)
GETREPORTURL="%s/api/getReport/" % (baseURL)
LOCATIONURL="%s/api/location/" % (baseURL)

In [2]:
def getHeaders():
  data={
      'username' : apiusername,
      'password' : apipassword
            }
  r=requests.post(AUTHENDPOINT,data=data)
  token=r.json()['token']
  headers={
      'content-type':'application/json',
      "Authorization" : "JWT " + token
    }
  return headers

In [3]:
def getAllState():
  locationArray=[]
  locationType='state'
  scheme='nrega'
  url=f"{LOCATIONURL}?locationType={locationType}&scheme={scheme}&limit=10000"
  r=requests.get(url)
  response=r.json()
  results=response['results']
  for res in results:
    dCode=res['code']
    locationArray.append(dCode)
  return locationArray

In [4]:
def getAllDistrict():
  locationArray=[]
  locationType='district'
  scheme='nrega'
  url=f"{LOCATIONURL}?locationType={locationType}&scheme={scheme}&limit=10000"
  r=requests.get(url)
  response=r.json()
  results=response['results']
  for res in results:
    dCode=res['code']
    locationArray.append(dCode)
  return locationArray

In [27]:
locationArray=getAllDistrict()
reportType="nicGlanceStats"
locationType='block'
statTypeSlug='persondays-generated-so-far'
finyear=19
#locationArray=["0212","0210"]
#locationArray=locationArray[:2]
blockStatDF=None
for locationCode in locationArray:
  print(locationCode)
  url=f"{REPORTURL}?reportType={reportType}&location__code={locationCode}"
  r=requests.get(url)
  results=r.json()
  count=results.get("count",None)
  if count == 1:
    reportURL=results['results'][0]['reportURL']
    df=pd.read_csv(reportURL)
    partDF=df[(df['finyear']==finyear) & (df['locationType']==locationType) & (df['slug']==statTypeSlug)]
    if blockStatDF is None:
      blockStatDF=partDF
    else:
      blockStatDF=pd.concat([blockStatDF, partDF], ignore_index=True)
blockStatDF.to_csv("blockStat_19_totalPersonDays.csv")

0212
0210
0211
0204
0207
0206
0213
0209
0208
0201
0203
0202
0205
0317
0310
0303
0307
0319
0316
0309
0318
0314
0304
0321
0312
0320
0301
0311
0315
0313
0305
0302
0306
0424
0405
0428
0403
0423
0430
0425
0408
0411
0401
0417
0420
0404
0414
0422
0429
0415
0407
0426
0419
0421
0402
0410
0432
0412
0413
0406
0416
0409
0431
0418
0427
0541
0551
0505
0545
0517
0527
0503
0548
0519
0507
0511
0550
0506
0549
0524
0525
0542
0546
0522
0520
0526
0515
0502
0508
0512
0501
0513
0523
0504
0521
0518
0509
0547
0543
0514
0510
0544
0516
3322
3319
3327
3311
3321
3317
3301
3312
3309
3303
3320
3314
3307
3310
3302
3323
3308
3306
3315
3325
3318
3313
3316
3304
3324
3326
3305
1112
1105
1122
1130
1108
1116
1104
1128
1129
1119
1132
1123
1111
1131
1101
1106
1107
1113
1110
1133
1127
1124
1125
1114
1120
1121
1102
1109
1117
1103
1126
1115
1118
1201
1213
1222
1209
1218
1210
1215
1220
1214
1204
1205
1203
1212
1219
1221
1217
1206
1211
1208
1216
1207
1202
1301
1302
1303
1304
1305
1306
1307
1308
1309
1310
1311
1312
1406
1404
1419


In [42]:
csvArray=[]
blockStatDF=pd.read_csv("data/blockStat_19_totalPersonDays.csv")
blockStatDF=blockStatDF[blockStatDF['value']>0]
blockStatDF.columns
allDistricts=blockStatDF.districtCode.unique()
print(f"Total nubmer of districts is {len(allDistricts)}")
selectedBlockArray=[]
#allDistricts=[3116]
for districtCode in allDistricts:
  #print(f"Processing District Code {districtCode}")
  df=blockStatDF[blockStatDF['districtCode']==districtCode]
  #print(f"length of df is {len(df)}")
  #print(len(df.value.unique()))
  bottomQuantileValue=df['value'].quantile(0.1)
  df=df[df['value']>=bottomQuantileValue]
  blockCodes=df['blockCode'].unique()
  selectedBlockCode=str(int(random.choice(blockCodes)))
  if len(selectedBlockCode) == 6:
    selectedBlockCode="0"+selectedBlockCode
  #print(selectedBlockCode)
  csvArray.append([selectedBlockCode])
  #selectedBlockArray.append(selectedBlockCode)
selectedDF=pd.DataFrame(csvArray,columns=["blockCode"])
print(selectedDF.head())
#selectedDF.to_csv("data/sampledBlocks.csv")

Total nubmer of districts is 656
  blockCode
0   0212016
1   0210001
2   0211009
3   0204018
4   0207030


In [ ]:
scheme='nrega'
reportType='panchayatReferenceDocument'
priority=90
startFinYear='18'
selectedBlocksDF=pd.read_csv("data/sampledBlocks_24Oct2019.csv")
selectedBlocksDF.head()
for index,row in selectedBlocksDF.iterrows():
  blockCode=str(int(row['blockCode']))
  if len(blockCode)==6:
    blockCode="0"+blockCode
  print(f"{655-index}-{blockCode}")
  url=f"{GETREPORTURL}?reportType=blockRejectedTransactions&startFinYear={startFinYear}&location__code={blockCode}&scheme={scheme}&priority=100"
  headers=getHeaders()
  r=requests.get(url,headers=headers)
  #print(r.json())  
  url=f"{LOCATIONURL}?locationType=panchayat&limit=10000&blockCode={blockCode}"
  r=requests.get(url)
  response=r.json()
  count=response.get("count",0)
  if count > 0:
    #print(f"the number of panchacyats is {count}")
    results=response['results']
    for res in results:
      panchayatCode=res.get("panchayatCode")
      #print(panchayatCode)
      url=f"{GETREPORTURL}?reportType={reportType}&startFinYear={startFinYear}&location__code={panchayatCode}&scheme={scheme}&priority={priority}"
      headers=getHeaders()
      r=requests.get(url,headers=headers)
      #print(r.json())
  

655-0212016
654-0210001
653-0211009
652-0204018
651-0207030
650-0206005
649-0213027
648-0209031
647-0208007
646-0201026
645-0203033
644-0202002
643-0205019
642-0317003
641-0310003
640-0303009
639-0307005
638-0316012
637-0316008
636-0309001
635-0311003
634-0314001
633-0304002
632-0309002
631-0312006
630-0307006
629-0301004
628-0311004
627-0315002
626-0313005
625-0305006
624-0302005
623-0306009
622-0424006
621-0405008
620-0409011
619-0403093
618-0423010
617-0416005
616-0425006
615-0408014
614-0411002
613-0401005
612-0417011
611-0420002
610-0404007
609-0414003
608-0422002
607-0413093
606-0415003
605-0407007
604-0426003
603-0419009
602-0421005
601-0402093
600-0410010
599-0415008
598-0412097
597-0413005
596-0406003
595-0416009
594-0409003
593-0401013
592-0418004
591-0427009
590-0541006
589-0551001
588-0505002
587-0545008
586-0517011
585-0527005
584-0503001
583-0548004
582-0519012
581-0507001
580-0511004
579-0550008
578-0506002
577-0549002
576-0524004
575-0525004
574-0542004
573-0546006
572-

In [32]:
locationArray=getAllState()
csvArray=[]
totalCount=0
totalSampleCount=0
headerColumns=["stateCode","totalBlocks","sampledBlocks"]
for locationCode in locationArray:
  url=f"{LOCATIONURL}?locationType=block&scheme=nrega&stateCode={locationCode}"
  r=requests.get(url)
  results=r.json()
  count=results.get("count",None)
  if count is not None:
    sampleCount=int(count*0.05)+1
    a=[locationCode,count,sampleCount]
    totalCount+=count
    csvArray.append(a)
    
    print(a)
df=pd.DataFrame(csvArray,columns=headerColumns)
print(df.head())
df.to_csv("data/totalSamplesPerState.csv")

['02', 661, 34]
['03', 115, 6]
['04', 239, 12]
['05', 534, 27]
['33', 146, 8]
['11', 248, 13]
['12', 140, 8]
['13', 80, 5]
['14', 323, 17]
['34', 264, 14]
['15', 176, 9]
['16', 152, 8]
['17', 313, 16]
['18', 354, 18]
['20', 70, 4]
['21', 46, 3]
['22', 26, 2]
['23', 74, 4]
['24', 314, 16]
['26', 150, 8]
['27', 295, 15]
['28', 32, 2]
['29', 385, 20]
['30', 58, 3]
['31', 824, 42]
['35', 95, 5]
['32', 342, 18]
['01', 9, 1]
['07', 1, 1]
['08', 2, 1]
['10', 12, 1]
['19', 10, 1]
['25', 3, 1]
['06', 3, 1]
['36', 438, 22]
  stateCode  totalBlocks  sampledBlocks
0        02          661             34
1        03          115              6
2        04          239             12
3        05          534             27
4        33          146              8


In [46]:
totalSampleDF=pd.read_csv("data/totalSamplesPerState.csv")
blockStatDF=pd.read_csv("data/blockStat_19_totalPersonDays.csv")

p={}
for index,row in totalSampleDF.iterrows():
  stateCode=str(row['stateCode'])
  if len(stateCode) == 1:
    stateCode="0"+stateCode
  sampledBlocks=row['sampledBlocks']
  p[stateCode]=sampledBlocks
  df=blockStatDF[blockStatDF['stateCode']==int(stateCode)]
  
  break


<bound method NDFrame.head of      Unnamed: 0  Unnamed: 0.1           state       district         block  \
0             0           162  ANDHRA PRADESH      ANANTAPUR         Agali   
1             1           310  ANDHRA PRADESH      ANANTAPUR      Amadagur   
2             2           458  ANDHRA PRADESH      ANANTAPUR    Amarapuram   
3             3           606  ANDHRA PRADESH      ANANTAPUR     Anantapur   
4             4           754  ANDHRA PRADESH      ANANTAPUR       Atmakur   
..          ...           ...             ...            ...           ...   
635         635          6230  ANDHRA PRADESH  WEST GODAVARI  Undrajavaram   
636         636          6378  ANDHRA PRADESH  WEST GODAVARI      Unguturu   
637         637          6526  ANDHRA PRADESH  WEST GODAVARI  Veeravasaram   
638         638          6674  ANDHRA PRADESH  WEST GODAVARI     Velairpad   
639         639          6822  ANDHRA PRADESH  WEST GODAVARI  Yelamanchili   

     panchayat  village  stateCod